In [34]:
with open('input') as f:
    data = [line.strip().split() for line in f]

In [68]:
from collections import Counter
from functools import cached_property, total_ordering


VALUES = "23456789TJQKA"

HAND_RANKS = (
    "High Card",
    "One Pair",
    "Two Pair",
    "Three of a Kind",
    "Full House",
    "Four of a Kind",
    "Five of a Kind",
)

class CamelCard:
    def __init__(self, value: str, *, joker: bool = False):
        assert len(value) == 1 and value in VALUES
        if joker:
            assert value != "J"
        self.value = value
        self.joker = joker

    def __repr__(self):
        return f"<Card object {self}>"

    def __str__(self):
        return self.value

    def __eq__(self, other):
        if self.joker or other.joker:
            return self.joker == other.joker
        return self.value == other.value

    def __gt__(self, other):
        return self.value_index > other.value_index

    @property
    def value_index(self):
        if self.joker:
            return 0
        return VALUES.index(self.value)


@total_ordering
class CamelCardsHand:
    def __init__(self, cards: list[CamelCard]):
        assert len(cards) == 5
        self.cards = cards

    @classmethod
    def from_str(cls, values: str, orig_values: str):
        assert len(values) == 5
        assert len(orig_values) == 5
        return cls([CamelCard(v, joker=o != v) for v, o in zip(values, orig_values)])

    def __repr__(self):
        return f"<CamelCardsHand object ({self})>"
    
    def __str__(self):
        card_strings = (str(card) for card in self)
        return "".join(card_strings)

    def __iter__(self):
        return iter(self.cards)

    def __eq__(self, other: "CamelCardsHand"):
        if self.rank_index == other.rank_index:
            return tuple(self) == tuple(other)
        return False

    def __gt__(self, other: "CamelCardsHand"):
        if self.rank_index == other.rank_index:
            return tuple(self) > tuple(other)
        return self.rank_index > other.rank_index

    @cached_property
    def rank(self):
        card_values = Counter([card.value for card in self])
        
        if len(card_values) == 1:
            return "Five of a Kind"
        if len(card_values) == 2:
            if 4 in card_values.values():
                return "Four of a Kind"
            return "Full House"
        if len(card_values) == 3:
            if 3 in card_values.values():
                return "Three of a Kind"
            return "Two Pair"
        if len(card_values) == 4:
            return "One Pair"
        return "High Card"
    
    @cached_property
    def rank_index(self):
        return HAND_RANKS.index(self.rank)

In [36]:
all_hands = [
    (CamelCardsHand.from_str(hand_str, hand_str), int(bid_str))
    for hand_str, bid_str in data
]

In [37]:
print("Part 1:")
print(sum(rank * bid for rank, (hand, bid) in enumerate(sorted(all_hands), start=1)))

Part 1:
253205868


In [38]:
from itertools import combinations_with_replacement

J_REPLACEMENTS = "23456789TQKA"

def get_best_hand_with_joker_replacement(orig_values):
    card_values = Counter(orig_values)
    if "J" in card_values:
        alt_hands = []
        num_jokers = card_values["J"]
        for replacements in combinations_with_replacement(J_REPLACEMENTS, r=num_jokers):
            alt_hand = list(orig_values)
            for replacement in replacements:
                j_pos = alt_hand.index("J")
                alt_hand[j_pos] = replacement
            alt_hand_str = "".join(alt_hand)
            alt_hands.append(CamelCardsHand.from_str(alt_hand_str, orig_values))
        
        best = max(alt_hands)
        return best
    return CamelCardsHand.from_str(orig_values, orig_values)

In [39]:
data = [
    ("32T3K", 765),
    ("T55J5", 684),
    ("KK677", 28),
    ("KTJJT", 220),
    ("QQQJA", 483),
]

In [ ]:
with open('input') as f:
    data = [line.strip().split() for line in f]

In [ ]:
all_hands_with_replaced_jokers = [
    (get_best_hand_with_joker_replacement(hand_str), int(bid_str))
    for hand_str, bid_str in data
]
print("Part 2:")
print(sum(rank * bid for rank, (hand, bid) in enumerate(sorted(all_hands_with_replaced_jokers), start=1)))

Part 2:
253938412


In [42]:
data[0]

['32555', '626']

In [43]:
# 253662843 too low
# 253938412 too high
# 254001790 too high

In [44]:
253938412-253662843

275569

In [45]:
hand = get_best_hand_with_joker_replacement("QQQJA")
tuple(hand)

(<Card object Q>,
 <Card object Q>,
 <Card object Q>,
 <Card object Q>,
 <Card object A>)

In [46]:
for rank, (hand, bid) in enumerate(sorted(all_hands_with_replaced_jokers), start=1):
    print(hand)

23KAT
246AQ
25K43
25K63
264KA
27TA5
27KQ4
285TQ
29Q85
29K63
2KA97
325Q8
327T8
35874
367Q4
367A5
37TA6
385K7
38K62
38A57
398KT
3TQA8
3QK69
3A2Q4
3A92Q
45QTA
46832
479TQ
49T38
4TK26
4Q8A6
4A358
4AT6K
528T4
53792
54632
564KA
564A8
56K28
56A34
5796K
57T32
5T694
5T7K9
5Q6K4
5Q92T
5K72A
6234Q
62493
628T7
62A9K
6358K
6385T
639K5
65792
73QT8
754TK
75832
7628A
7629A
7658A
7896Q
79236
792T6
7935A
795T8
7K523
7AQ25
823T4
87234
876K4
87A54
8T43K
8T47K
8TA65
8K492
8K596
92T37
935Q6
943T7
95863
95Q6T
96A52
97645
9T27Q
9T7Q6
9Q5TK
9Q83A
T463A
T5687
T7K5Q
TQ457
TQ7KA
TQ985
TQK76
TK689
TK8Q2
Q3986
Q4739
Q5A49
Q6739
Q6T2K
Q7A69
QT246
QK495
QKT3A
QKT65
K2853
K2A8T
K36A9
K36AQ
K3T9A
K49QT
K572T
K8TQ7
K938A
KT539
KT6A7
A2TK7
A3249
A37T2
A4968
A4T9Q
A4Q5K
A56K9
A6T2Q
A7K62
A8Q45
AT368
AQ9K6
AKQT5
27793
29T2Q
22845
667KQ
37538
48469
797TK
2T327
4T4A6
3T836
22798
243TT
2993T
2Q6TQ
2A2T9
45A64
26T32
373T8
4754T
2A84A
22QA4
55T79
363A9
8K98Q
6KT6A
5A5T8
328T8
33947
32A26
36TQ3
36K8K
38436
38T34
3T242
3T3K7
3T32

In [71]:
get_best_hand_with_joker_replacement("JJJJJ")

<CamelCardsHand object (22222)>

In [53]:
CamelCardsHand.from_str("AAAAA", "JJJJJ").rank_index

6

In [69]:
hand_1 = CamelCardsHand.from_str("KKKKK", "JKKKK")
hand_2 = CamelCardsHand.from_str("KKKK2", "KKKK2")
cards = [hand_1, hand_2]
assert hand_1 > hand_2

In [72]:
max(cards)